# Coursework 3

Problems are set by Dr. Wei Dai, 2022

Check the lecture notes for the steps of ADMM and the stopping criteria. 

In [29]:
using DataFrames, FFTW, Wavelets, Images, LinearAlgebra, Random, StatsBase

# Make sure that you complete the involvement table. 
# The first row is for CID number. 
# Other rows are for the involvement for each "big" problem (8 big problems in coursework 1). 
# "1" for getting involved in this part. 
# "0" for not involved
Contributions = DataFrame( A=[01700345,1,1,1], B = [01518690,1,1,1], C = [02243990,1,1,1], D = [01708685,1,1,1] )

Row,A,B,C,D
,Int64,Int64,Int64,Int64
1,1700345,1518690,2243990,1708685
2,1,1,1,1
3,1,1,1,1
4,1,1,1,1


## 1. Netflix Problem (40%)

The $\texttt{ratings.csv}$ is MovieLens data downloaded from [GroupLens](grouplens.org).

### 1.1 Prepare the data (6%)

1. Use the package $\texttt{xlsx.jl}$ or $\texttt{csv.jl}$ to read $\texttt{ratings.csv}$. Using the existing movie rating data to form a matrix $\bm{X}_0$, where the rows correspond to users and the columns correspond to movies. Note that for our purposes, we delete all the movies that have less than 15 ratings.$\newline$ Show the size of your matrix $\bm{X}_0$. Denote its size by $M \times N$.        

2. Based on your obtained $\bm{X}_0$, define 
        $$\begin{align*}
            \Delta & := \left\{(m,n):~ \text{a rating exists for the $m$-th user and $n$-th movie}  \right\}. 
        \end{align*}$$
        In Julia, we use $\texttt{Delta = Array{Int64,2}(undef,0,2)}$ to initialize $\Delta$. $\newline$ Find $\Delta$ ($\texttt{Delta}$ in Julia). Show the size of $\texttt{Delta}$. 
        
3. Use $\texttt{Random.seed!(1234)}$ to initialize the seed for random number generator. Use $\texttt{StatsBase.sample}$ to sample (without replacement) 80\% of the entries in $\texttt{Delta}$ to form $\texttt{Omega}$ ($\Omega$). (Use $\texttt{ceil}$ function to calculate the size of $\Omega$.)
$\newline$
Write a function $\texttt{pOmega}$ to obtain $\bm{y} = \mathcal{P}_{\Omega}(\bm{X}_0)$. 
$\newline$
The vector $\bm{y}$ contains our training data. The vector $\bm{y}_e = \mathcal{P}_{\Delta \backslash \Omega}(\bm{X}_0)$ gives the test data for performance evaluation. 
$\newline$
Find both $\bm{y}$ and $\bm{y}_e$. 


In [ ]:
######1.1.1########
using CSV, DataFrames
cs = CSV.read("ratings.csv", DataFrame)
X0 = zeros(610,193609);
for i in 1:100836
    X0[cs[i,1],cs[i,2]] = cs[i,3]
end
count = 0;
arr = [];
for i in 1:193609
    for j in 1:610
        if X0[j,i] != 0
            count = count + 1;
        end
    end
    if count>=15
        push!(arr,i);
    end
    count = 0;
end
Y = Array{Int64,2}(undef,610,0);
old=0;
for q in arr
    Y = [Y X0[:,q]]
end
X0 = copy(Y);


In [ ]:
######1.1.2######
(M,N) = size(X0)
Delta = Array{Int64,2}(undef,0,2);
    for i in 1:M
        for j in 1:N
            if X0[i,j] != 0
                d = [i j;]
                Delta = [Delta;d]
            end
        end
    end

In [ ]:
######1.1.3#######
using LinearAlgebra
using BenchmarkTools
using Base
using Random
using Pkg
Pkg.add("StatsBase")
using StatsBase
Random.seed!(1234)
size_Omega = ceil(Int, 0.8*length(Delta[:,1]));
Omega = sample(1:length(Delta[:,1]),size_Omega;replace=false);
Omega = sort(Omega); 
function pOmega(X0,Delta,Omega)
    y = [];
    for i in Omega
        y = [y;X0[Delta[i,1],Delta[i,2]]]
    end
    return y;
end
y = pOmega(X0,Delta,Omega);
j = 1;
del = [];
for i in 1:length(Delta[:,1])
    if Omega[j] == i
        j = j + 1;
        continue
    end
    push!(del,i);
end
ye = pOmega(X0,Delta,del);

### 1.2 Employ alternating optimization algorithm (10%)

In this part, we are solving $$\begin{align*}
        & \min_{\bm{X}}~ \delta\left( \text{rank}(\bm{X}) \le 10 \right)
        + \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2
    \end{align*}$$
using alternating minimization. More specifically, let $\bm{L} \in \mathbb{R}^{M \times 10}$ and $\bm{R} \in \mathbb{R}^{N \times 10}$. We solve 
$$\begin{align*}
    & \min_{\bm{L},\bm{R}}~ \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{L} \bm{R}^{\mathsf{T}}) \rVert_2^2
\end{align*}$$
in an alternating way.

1. Implement the alternating minimization algorithm. Test it with randomly initialized $\bm{L}$ and $\bm{R}$ using $\texttt{randn(Float64,M,10)}$ and $\texttt{randn(Float64,N,10)}$ respectively. Denote the output of your alternating minimization algorithm $\hat{\bm{X}}$. 

2. Find the training performance $\lVert \mathcal{P}_{\Omega} (\hat{\bm{X}} - \bm{X}_0)\rVert_2^2 / |\Omega|$ and the prediction performance $\lVert \mathcal{P}_{\Delta \backslash \Omega} (\hat{\bm{X}} - \bm{X}_0)\rVert_2^2 / |\Delta \backslash \Omega|$. 

In [ ]:
#####1.2.1#####
(M,N) = size(X0);
P = randn(M,10);
Q = randn(N,10);
for k in 1:3
    for j in 1:N
        oj = [];
        for p in 1:M
            if X0[p,j] != 0
                oj = [oj;p];
            end
        end
        Q[j,:] = pinv(ones(1,10)*transpose(P[oj,:])*P[oj,:])*ones(1,10)*transpose(P[oj,:])*X0[oj,j];
    end
    for i in 1:M
        oi = [];
        for q in 1:N
            if X0[i,q] != 0
                oi = [oi;q];
            end
        end
        P[i,:] = pinv(Q[oi,:]*ones(10,1))*Q[oi,:]*ones(10,1)*reshape(X0[i,oi],1,length(X0[i,oi]))*pinv(transpose(Q[oi,:]));
    end
end
X_hat = P*Q';
@show (norm(X_hat-X0))

In [ ]:
#####1.2.2######
performance_tra = norm(pOmega(X_hat,Delta,Omega))/sqrt(norm(Omega));
performance_per = norm(pOmega(X_hat,Delta,del))/sqrt(norm(del));
println(performance_tra)
println(performance_per)

### 1.3 Employ proximal gradient method (10%)

In this part, we are solving $$\begin{align*}
        & \min_{\bm{X}}~ \lambda \lVert \bm{X} \rVert_*
        + \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2
    \end{align*}$$
    where $\lambda>0$ is chosen so that your result $\hat{\bm{X}}$ has a rank close to 10 (all the singular values except the first 10 ones are close to 10). More specifically, use the proximal gradient method to solve the above convex optimization problem. 

1. Give the closed form of the gradient of the term $\frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2$ in terms of $\mathcal{P}_{\Omega}$ and $\bm{X}$.
        
2. Implement the proximal gradient method. 
        
3. Find an appropriate value for $\lambda >0$. Test the training performance and the prediction performance.

1.
$\nabla \frac{1}{2} \left| \left| y-P_{\Omega }\left( X\right)  \right|  \right|^{2}_{2} $ 

$=-P^{\ast }_{\Omega }\left( y-P_{\Omega }(X)\right)  $

$=-P^{\ast }_{\Omega }(y)+P^{\ast }_{\Omega }\left( P_{\Omega }\left( X\right)  \right)  $



In [ ]:
#######1.3.2#######
function invpOmega(x,Delta,Omega,M,N)
    X1 = zeros(M,N)
    r = 1;
    for i in Omega
        X1[Delta[i,1],Delta[i,2]] = x[r]
        r = r + 1
    end
    return X1;
end
lamda = 0.2;
gama = 1;
yita = 0.5;
oldx = randn(M,N);
z = 0;
qian = 100;
X_s = zeros(M,N);
#while qian+lamda*norm(W0,1)>0.1
for jj in 1:5
    newgama = gama;
    while true
        z = oldx + yita*invpOmega((y - pOmega(oldx,Delta,Omega)),Delta,Omega,M,N)
        sigma, u, v = svd(z);
        X_s = zeros(M,N);
        for i in 1:min(M,N)
            sigma_i = sigma[:,i];
            s_i = copy(sigma_i);
            ui = u[i];
            vi = v[:,i];
            for j in 1:length(sigma_i)
                s_i[j] = max(0,sigma_i[j]-lamda*gama);
            end
            X_s = X_s + s_i*ui*vi';
        end
        qian = 0.5*sum((y - pOmega(X_s,Delta,Omega)).^2);
        hou = 0.5*sum((y - pOmega(oldx,Delta,Omega)).^2)+vec(X_s-oldx)'*vec(z)+1/newgama*0.5*sum((vec(X_s-oldx)).^2);
        println(hou);
        if abs(qian)<=abs(hou)
           break;
        end
        # # #######find a valid step size#######
        # newgama = yita*newgama;
    end
    gama = newgama;
    oldx = X_s;
end 

In [ ]:
performance_tra = norm(pOmega(X_s,Delta,Omega))/sqrt(norm(Omega));
performance_per = norm(pOmega(X_s,Delta,del))/sqrt(norm(del));
println(performance_tra)
println(performance_per)

### 1.4 Employ ADMM (14%)

A more practical approach is to solve the following constrained optimization problem 
    $$\begin{align*}
        \min_{\bm{X}}~
        & \lambda \lVert \bm{X} \rVert_*
        + \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2 \\
        \text{s.t.}~
        & 0 \le X_{m,n} \le 5,~ \forall~1 \le m \le M,~ 1 \le n \le N. 
    \end{align*}$$
We simplify the notation by writing $0 \le \bm{X} \le 5$ for the constraint.

We use ADMM to solve the constrained optimization problem. An ADMM Lagrangian is given by 
$$\begin{align*}
    \min_{\bm{X},\bm{Z}}~ 
    & \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}(\bm{X}) \rVert_2^2 
    + \lambda \lVert \bm{Z}_1 \rVert_*
    + \delta \left( 0 \le \bm{Z}_2 \le 5 \right) \\
    & + \left\langle \bm{\Lambda}_1,\bm{X}-\bm{Z}_1 \right\rangle
    + \left\langle \bm{\Lambda}_2,\bm{X}-\bm{Z}_2 \right\rangle \\
    & + \frac{\rho}{2} \lVert \bm{X}-\bm{Z}_1 \rVert_F^2 
    + \frac{\rho}{2} \lVert \bm{X}-\bm{Z}_2 \rVert_F^2. 
\end{align*}$$

1. Find the closed form for minimization with respect to $\bm{X}$ for given $\bm{Z}$ and $\bm{\Lambda}$.

2. Find the closed form for minimization with respect to $\bm{Z}$ for given $\bm{X}$ and $\bm{\Lambda}$.         

3. Implement the ADMM procedure. 

4. With properly chosen $\lambda > 0$, test the training performance and the prediction performance. 

$B=\left[ \begin{matrix}I\\ I\end{matrix} \right]  $

$D=\left[ \begin{matrix}-I&\\ &-I\end{matrix} \right]  $

$Z=\left[ \begin{matrix}Z_{1}\\ Z_{2}\end{matrix} \right]  $

$\Lambda =\left[ \begin{matrix}\Lambda_{1} \\ \Lambda_{2} \end{matrix} \right]  $

$BX+DZ=0$

$\begin{aligned} & X^{k+1}=\underset{X}{\operatorname{argmin}} \frac{1}{2}\|y-P_{\Omega }(X)\|_2^2+\frac{\rho}{2}\left\|BX^{k}+DZ^{k}+\frac{\Lambda^k}{\rho}\right\|_2^2 \\ 
& Z^{k+1}=\underset{Z_1, Z_2}{\operatorname{argmin}} \lambda\left\|Z_1\right\|_*+\frac{\rho}{2}\left\|X^{k+1}-Z_1+\frac{\Lambda_1^k}{\rho}\right\|_2^2 \\
& +\delta\left(0 \leq Z_2 \leq 5\right)+\frac{\rho}{2}\left\|X^{k+1}-Z_2+\frac{\Lambda_2^k}{\rho}\right\|_2^2 \\ 
& \Lambda^{k+1}=\Lambda^{k}+P\left(B X^{k+1}+D Z^{k+1}\right) \end{aligned}$

1.4.1

$X^{k+1}=\left( \Omega^{T} \Omega +\rho B^{T}B\right)^{-1}  \left( \Omega^{T} y-\rho B^{T}DZ-B^{T}\Lambda^{k} \right)  $

1.4.2

$\begin{aligned} & X^{k+1}+\frac{\Lambda_1^k}{\rho}=\sum_{i=1}^{\min (m, n)} \sigma_i u_i v_i^{\top} \\ & Z_1^{k+1}=\sum_{i=1}^{\min (m, n)} \eta\left(\sigma_i ; \frac{\lambda}{\rho}\right) u_i v_i^{\top} \end{aligned}$

$Z^{k+1}_{2}=max(min(X^{k+1}+\frac{\Lambda^{k}_{2} }{\rho },0),5) $

In [59]:
#####1.4.3/4#####
function eye(n)
    ma = zeros(n,n);
    for i in 1:n
        ma[i,i] = 1;
    end
    return ma
end

eye (generic function with 1 method)

In [ ]:
Delta2 = Array{Int64,2}(undef,0,2);;
j = 1;
for i in 1:1650
    while j <= 73845
    if Delta[j,2] == i
        #reshape(Delta[j,:],(1,2))
        Delta2 = [Delta2;reshape(Delta[j,:],(1,2))]
    end
    j = j + 1
    end
    j = 1;
end

In [61]:
function XK1(Omega,Delta,z1,z2,v1,v2,B,D,X0)
    j = 1;
    id = [];
    Xk1 = zeros(size(X0))
    for i in Omega
        if Delta[i,2] == j
            push!(id,Delta[i,1]);
            continue
        end
        y = zeros(length(id),0);
        x = X0[:,j];
        om = zeros(length(id),length(x));
        for k in 1:length(id)
            om[k,id[k]] = 1;
        end
        y = om*x;
        ########
        z = [z1[:,j];z1[:,j]]
        v = [v1[:,j];v2[:,j]]
        Xk1[:,j] = pinv(om'*om .+ rou.*B'*B)*(om'*y-rou.*B'*D*z-B'*v)
        j = j + 1;
        id = [];
        if Delta[i,2] == j
            push!(id,Delta[i,1]);
        end
    end
    return Xk1
    end
    #@show y

XK1 (generic function with 1 method)

In [ ]:
rou = 1;
lamda = 0.1;
(M,N) = size(X0);
B = [eye(M);eye(M)]
D = [-eye(M) zeros(M,M);zeros(M,M) -eye(M)]
z1 = ones(M,N)
z2 = ones(M,N)
v1 = ones(M,N)
v2 = ones(M,N)
for l in 1:10
    Xk1 = XK1(Omega,Delta2,z1,z2,v1,v2,B,D,X0)
    sigma, u, v = svd(Xk1+v1./rou);
    Z1 = zeros(M,N);
    for i in 1:min(M,N)
        sigma_i = sigma[:,i];
        s_i = copy(sigma_i);
        ui = u[i];
        vi = v[:,i];
        for j in 1:length(sigma_i)
            s_i[j] = max(0,sigma_i[j]-1/rou*lamda);
        end
        Z1 = Z1 + s_i*ui*vi';
    end
    z1 = Z1;
    z2 = min.(max.(Xk1+v1./rou,0),5)
    V = [v1;v2] + rou*(B*Xk1+D*[z1;z2])
    v1 = V[1:M,1:N]
    v2 = V[M+1:2M,1:N]
    @show norm(Xk1-X0)
end

## 2. Video Background and Foreground Separation (20%)

In this application, we use the video file $\texttt{demo.avi}$.

### 2.1 Data preparation (5%)

The goal of this step is to form a matrix where each column is a vectorized version of a frame of the video and the number of columns is the number of the frames in the video. Denote the matrix by $\bm{Y}$.

With help from GTA Mr. Zhengang Guo, we have identified that the function $\texttt{VideoIO.load}$ can load the video file and that the function $\texttt{VideoIO.channelview}$ can get frames from the loaded video. 

Write a function $\texttt{video2matrix}$ to form the video matrix $\bm{Y}$ from the video file. 


In [68]:
using VideoIO
function video2matrix(video_path)
    temp = VideoIO.load(video_path)
    N, = size(temp)
    Y = Array{Float64}(undef, 2304,51)
    for i in 1:N
     Y[:,i] = Float64.(vec(VideoIO.channelview(temp[i])[1,:,:]))
    end
    return Y
end
Vid = video2matrix("demo.avi");

### 2.2 Employ ADMM (15%)

The video matrix $\bm{Y}$ can be modelled as 
    $$\begin{align*}
        & \bm{Y} = \bm{L} + \bm{S} + \bm{W},
    \end{align*}$$
where $\bm{L}$ is a low rank matrix to represent the background, $\bm{S}$ is a sparse matrix (only a few entries are nonzero) that correspond to moving foreground, and $\bm{W}$ denotes Gaussian noise.

We consider the following constrained optimization problem 
$$\begin{align*}
    \min_{\bm{L},\bm{S}}~ 
    & \frac{1}{2} \lVert \bm{Y} - \bm{L} -\bm{S} \rVert_F^2
    + \lambda_1 \lVert \bm{L} \rVert_* 
    + \lambda_2 \lVert \bm{S} \rVert_1 \\
    \text{s.t.}~
    & 0 \le \bm{L} \le 255,  
\end{align*}$$
where with slight abuse of notation $\lVert \bm{S} \rVert_1 := \lVert \text{vect}(\bm{S}) \rVert_1$. 
    An ADMM optimization formulation is given below. 
$$\begin{align*}
    \min_{\bm{L},\bm{S}, \bm{Z}}~ 
    & \frac{1}{2} \lVert \bm{Y} - \bm{L} -\bm{S} \rVert_F^2
    + \lambda_1 \lVert \bm{Z}_1 \rVert_* 
    + \delta \left( 0 \le \bm{Z}_2 \le 255 \right)
    + \lambda_2 \lVert \bm{Z}_3 \rVert_1 \\
    \text{s.t.}~
    & \bm{L} = \bm{Z}_1,~
    \bm{L} = \bm{Z}_2,~
    \bm{S} = \bm{Z}_3. 
\end{align*}$$

1. Find the closed form for minimization with respect to $(\bm{L},\bm{S})$ for given $\bm{Z}$ and $\Lambda$. 

2. Find the closed form for minimization with respect to $\bm{Z}$ for given $(\bm{L},\bm{S})$ and $\Lambda$. 

3. Implement the corresponding ADMM procedure. 

4. Choose reasonable parameters $\lambda_1$ and $\lambda_2$ to have nice results. Save the estimated $\bm{L}$ and $\bm{S}$ into video files $\texttt{Background.avi}$ and $\texttt{Foreground.avi}$ respectively. (Upload these files when you submit your coursework.)

$B=\left[ \begin{matrix}I\\ I\end{matrix} \right]  $

$D=\left[ \begin{matrix}-I&\\ &-I\end{matrix} \right]  $

$Z=\left[ \begin{matrix}Z_{1}\\ Z_{2}\end{matrix} \right]  $

$\Lambda_{12} =\left[ \begin{matrix}\Lambda_{1} \\ \Lambda_{2} \end{matrix} \right]  $

$BX+DZ=0$

$\begin{aligned} & L^{k+1}=\underset{L}{\operatorname{argmin}}\left(\frac{1}{2}\left\|Y-L-S^k\right\|_F^2+\frac{\rho}{2}\left\|B L+D Z^k+\frac{\Lambda_{12}^k}{P}\right\|_2^2\right) \\ & S^{k + 1}=\underset{S}{\operatorname{argmn}}\left(\frac{1}{2}\left\|Y-L^{k+1}-S\right\|_F^2+\frac{\rho}{2}\left\|S-Z_3^k +\frac{\Lambda_3^k}{\rho}\right\|_2^2\right) \\ 
& Z_1^{k+1}=\underset{Z_1}{\operatorname{argmin}} \lambda_1\left\|Z_1\right\|_*+\frac{\rho}{2}\left\|L^{k+1}-Z_1+\frac{\Lambda_1^k}{\rho}\right\|_2^2 \\ & Z_2^{k+1}=\underset{Z_2}{\operatorname{argmin}} \delta\left(0 \leq Z_2 \leq 255\right)+\frac{p}{2}\left\|L^{k+1}-Z_2+\frac{\Lambda_2^k}{\rho}\right\|_2^2 \\ & \end{aligned}$
$\begin{aligned} & Z_3^{k+1}=\underset{Z_3}{\operatorname{argmin}} \cdot \lambda_2\left\|Z_3\right\|_1+\frac{\rho}{2}\left\|S^{k+1}-Z_3+\frac{\Lambda_3^k}{\rho}\right\|^2 \\ & \Lambda_{12}^{k+1}=\Lambda_{12}+\rho\left(B L^{k+1}+D Z^{k+1}\right) \\ & \Lambda_3^{k+1}=\Lambda_3+\rho\left(S_{k+1}-Z_3^{k+1}\right) \end{aligned}$

2.2.1

$L^{k+1}=\left(I +\rho B^{T}B\right)^{-1}  \left(Y-S^k-\rho B^{T}DZ^k-B^{T}\Lambda_{12}^{k} \right)  $

$S^{k+1}=\left( 1+\rho \right)^{-1}  \left( Y-L^{k+1}+\rho Z_{3}-\Lambda^{k}_{3} \right)  $

2.2.2

$\begin{aligned} & L^{k+1}+\frac{\Lambda_1^k}{\rho}=\sum_{i=1}^{\min (m, n)} \sigma_i u_i v_i^{\top} \\ & Z_1^{k+1}=\sum_{i=1}^{\min (m, n)} \eta\left(\sigma_i ; \frac{\lambda_1}{\rho}\right) u_i v_i^{\top} \end{aligned}$

$Z^{k+1}_{2}=min(max(L^{k+1}+\frac{\Lambda^{k}_{2} }{\rho },0),5) $

$Z_3^{k+1}=max(S^{k+1}+\frac{\Lambda^{k}_{3} }{\rho },0) $

In [ ]:
####2.2.3/4#####
rou = 1;
lamda1 = 0.1;
lamda2 = 0.1;
(M,N) = size(Vid);
B = [eye(M);eye(M)]
D = [-eye(M) zeros(M,M);zeros(M,M) -eye(M)]
L = ones(M,N)
S = ones(M,N)
z1 = ones(M,N)
z2 = ones(M,N)
z3 = ones(M,N)
v1 = ones(M,N)
v2 = ones(M,N)
v3 = ones(M,N)
for l in 1:10
    z = [z1;z2]
    v = [v1;v2]
    L = pinv(1 .+ rou.*B'*B)*(Vid - S - rou.*B'*D*z-B'*v)
    S = 1/(1 + rou).*(Vid - L +rou.*z3 .- v3)
    sigma, u, v = svd(L+v1./rou);
    Z1 = zeros(M,N);
    for i in 1:min(M,N)
        sigma_i = sigma[:,i];
        s_i = copy(sigma_i);
        ui = u[i];
        vi = v[:,i];
        for j in 1:length(sigma_i)
            s_i[j] = max(0,sigma_i[j]-1/rou*lamda);
        end
        Z1 = Z1 + s_i*ui*vi';
    end
    z1 = Z1;
    z2 = min.(max.(L+v2./rou,0),1)
    V = [v1;v2] + rou*(B*L+D*[z1;z2])
    v1 = V[1:M,1:N]
    v2 = V[M+1:2M,1:N]
    v3 = v3 + rou.*(S - z3)
    @show norm(Vid-L)
end

## 3. MRI Compressed Sensing Recovery (40%)

The following optimization problem can be formulated for MRI compressed sensing recovery. 
$$\begin{align*}
    \min_{\bm{X}}~
    & \frac{1}{2} \lVert \bm{y} - \mathcal{P}_{\Omega}\left( \mathcal{F} \left( \bm{X} \right) \right) \rVert_2^2
    + \alpha \lVert \mathcal{H}(\bm{X}) \rVert_1\\
    & + \beta \lVert \mathcal{D}_1(\bm{X}) \rVert_1
    + \beta \lVert \mathcal{D}_2(\bm{X}) \rVert_1
    + \beta \lVert \mathcal{D}_3(\bm{X}) \rVert_1
    + \beta \lVert \mathcal{D}_4(\bm{X}) \rVert_1 \\
    \text{s.t.}~
    & 0 \le \bm{X} \le 255,
\end{align*}$$
where $\mathcal{H}$ stands for the discrete Haar wavelet transform (See the section Haar matrix at [Wikipedia](https://en.wikipedia.org/wiki/Haar_wavelet). $\mathcal{D}_1(\bm{X})$, $\mathcal{D}_2(\bm{X})$, $\mathcal{D}_3(\bm{X})$, and $\mathcal{D}_4(\bm{X})$ compute $X_{m,n} - X_{m-1,n}$, $X_{m,n} - X_{m,n-1}$, $X_{m,n} - X_{m-1,n-1}$, $X_{m,n} - X_{m-1,n+1}$, respectively, for all valid $m,n$ to form the corresponding vetors.


### 3.1 (5%)
Find the form of the adjoint operators for $\mathcal{P}_{\Omega}(\mathcal{F}(\cdot))$, $\mathcal{H}$, $\mathcal{D}_1$, $\mathcal{D}_2$, $\mathcal{D}_3$, and $\mathcal{D}_4$.    
    

### 3.2 (5%)

Find an ADMM optimization formulation so that each ADMM step can be solved using proximal operators with closed form solutions. 

### 3.3 (5%)

Find the corresponding ADMM Lagrangian. Note that complex numbers are involved in the formulation due to the Fourier transform. The inner products of Lagrange multipliers and variables are in the form of $\Re \left(\langle \bm{\lambda},\bm{x} \rangle\right) $ where $\Re$ denotes the real part of the number.

### 3.4 (5%)

Find the closed form solutions for the ADMM steps. Write your solutions using the operators and the adjoint operators.

### 3.5 (20%)

Implement an ADMM and use the data in the previous coursework to test the performance. 

1. Write functions (or use existing functions in Julia) for the involved operators $\mathcal{P}_{\Omega}(\mathcal{F}(\cdot))$, $\mathcal{H}$, $\mathcal{D}_1$, $\mathcal{D}_2$, $\mathcal{D}_3$, $\mathcal{D}_4$, and their adjoint operators. Note that the explicit matrix forms will lead to large storage space and slow computation speed. We expect that you do not use the explicit matrix forms. (5%)

2. Write functions to solve the involved proximal operators and least squares problems. As the involved linear operators are implemented using functions (not matrices), the conjugate gradient (CG) method are needed. The description of the CG method can be found from Wikipedia. A Julia implementation of can be found from the package $\texttt{IterativeSolvers.jl}$. You will also need $\texttt{LinearMaps.jl}$ as our linear operators are implemented as functions rather than matrices. (10%)

3. ADMM implementation and test. (5%)

3.1

$F^{\ast }\left( P^{\ast }_{\Omega }\left( \cdot \right)  \right)  $

${}^{H^{\ast }\left( \cdot \right)  }$ $D^{\ast }_{1}\left( \cdot \right)  $
$D^{\ast }_{2}\left( \cdot \right)  $
$D^{\ast }_{3}\left( \cdot \right)  $
$D^{\ast }_{4}\left( \cdot \right)  $

3.2

$min \frac{1}{2} \left| \left| y-P_{\Omega }\left( F\left( X\right)  \right)  \right|  \right|^{2}_{2}  +\alpha \left| \left| H\left( Z_{5}\right)  \right|  \right|_{1}  +\delta \left( 0\leqslant Z_{6}\leqslant 255\right)  +\beta \left| \left| D_{1}\left( Z_{1}\right)  \right|  \right|_{1}  +\beta \left| \left| D_{2}\left( Z_{2}\right)  \right|  \right|_{1}  +\beta \left| \left| D_{3}\left( Z_{3}\right)  \right|  \right|_{1}  +\beta \left| \left| D_{4}\left( Z_{4}\right)  \right|  \right|_{1}  $

s.t. BX+DZ=0

$B=\left[ \begin{matrix}I\\ I\\ I\\ I\\ I\\ I\end{matrix} \right]  $

$D=\left[ \begin{matrix}-I&&&&&\\ &-I&&&&\\ &&-I&&&\\ &&&-I&&\\ &&&&-I&\\ &&&&&-I\end{matrix} \right]  $

$Z=\left[ \begin{matrix}Z_{1}\\ Z_{2}\\ Z_{3}\\ Z_{4}\\ Z_{5}\\ Z_{6}\end{matrix} \right]  $
$\Lambda =\left[ \begin{matrix}\Lambda_{1} \\ \Lambda_{2} \\ \Lambda_{3} \\ \Lambda_{4} \\ \Lambda_{5} \\ \Lambda_{6} \end{matrix} \right]  $

\begin{aligned} & X^{k+1}=\underset{X}{\operatorname{argmin}} \frac{1}{2}\left\|y-P_\Omega (F(X))\right\|_2^2+\frac{\rho}{2}\left\|B X+D Z^k+\frac{\Lambda^k}{\rho}\right\|_2^2 \\ & Z_5^{k+1}=\underset{Z_5}{\operatorname{argmin}} \alpha\left\|H\left(Z_5\right)\right\|_1+\frac{\rho}{2}\left\|X^{k+1}-Z_5+\frac{\Lambda_5^k}{\rho}\right\|_2^2 \\ & Z_6^{k+1}=\underset{Z_6}{\operatorname{argmin}} \delta\left(0 \leq Z_6 \leq 255\right)+\frac{\rho}{2}\left\|X^{k+1}-Z_6+\frac{\Lambda_6^k}{\rho}\right\|_2^2 \\ & Z^{k+1}_{1}=\underset{Z_1}{\operatorname{argmin}}\beta \left| \left| D_{1}\left( X^{k+1}\right)  \right|  \right|_{1}  +\frac{\rho }{2} \left| \left| X^{k+1}-Z_{1}+\frac{\Lambda^{k}_{1} }{\rho } \right|  \right|^{2}_{2}   \\ & Z^{k+1}_{2}=\underset{Z_2}{\operatorname{argmin}}\beta \left| \left| D_{2}\left( X^{k+1}\right)  \right|  \right|_{1}  +\frac{\rho }{2} \left| \left| X^{k+1}-Z_{2}+\frac{\Lambda^{k}_{2} }{\rho } \right|  \right|^{2}_{2}   \\ & Z^{k+1}_{3}=\underset{Z_3}{\operatorname{argmin}}\beta \left| \left| D_{3}\left( X^{k+1}\right)  \right|  \right|_{1}  +\frac{\rho }{2} \left| \left| X^{k+1}-Z_{3}+\frac{\Lambda^{k}_{3} }{\rho } \right|  \right|^{2}_{2}   \\ & Z^{k+1}_{4}=\underset{Z_4}{\operatorname{argmin}}\beta \left| \left| D_{4}\left( X^{k+1}\right)  \right|  \right|_{1}  +\frac{\rho }{2} \left| \left| X^{k+1}-Z_{4}+\frac{\Lambda^{k}_{4} }{\rho } \right|  \right|^{2}_{2}   \\ & \Lambda^{k+1}=\Lambda^k+\rho\left(B X^{k+1}+D Z^{k+1}\right) \end{aligned}

3.3

$L\left( X,Z,\Lambda \right)  =\frac{1}{2} \left| \left| y-P_{\Omega }\left( F\left( X\right)  \right)  \right|  \right|^{2}_{2}  +\alpha \left| \left| H\left( Z_{5}\right)  \right|  \right|_{1}  +\delta \left( 0\leqslant Z_{6}\leqslant 255\right)  +\beta \left| \left| D_{1}\left( Z_{1}\right)  \right|  \right|_{1}  +\beta \left| \left| D_{2}\left( Z_{2}\right)  \right|  \right|_{1}  +\beta \left| \left| D_{3}\left( Z_{3}\right)  \right|  \right|_{1}  +\beta \left| \left| D_{4}\left( Z_{4}\right)  \right|  \right|_{1}  +\Re \left( \langle \Lambda_{1} ,X-Z_{1}\rangle \right)  +\Re \left( \langle \Lambda_{2} ,X-Z_{2}\rangle \right)  +\Re \left( \langle \Lambda_{3} ,X-Z_{3}\rangle \right)  +\Re \left( \langle \Lambda_{4} ,X-Z_{4}\rangle \right)  +\Re \left( \langle \Lambda_{5} ,X-Z_{5}\rangle \right)  +\Re \left( \langle \Lambda_{6} ,X-Z_{6}\rangle \right)  +\frac{\rho }{2} \left| \left| X-Z_{1}\right|  \right|^{2}_{2}  +\frac{\rho }{2} \left| \left| X-Z_{2}\right|  \right|^{2}_{2}  +\frac{\rho }{2} \left| \left| X-Z_{3}\right|  \right|^{2}_{2}  +\frac{\rho }{2} \left| \left| X-Z_{4}\right|  \right|^{2}_{2}  +\frac{\rho }{2} \left| \left| X-Z_{5}\right|  \right|^{2}_{2}  +\frac{\rho }{2} \left| \left| X-Z_{6}\right|  \right|^{2}_{2}  $

3.4

$\begin{aligned} & -F^*\left(P_{\Omega }^*(X)\right)\left(y-P_{\Omega }(F(X))\right)+\rho B^{\top}\left(B X+D Z^k+\frac{\Lambda}{\rho}^k\right)=0 \\ & \alpha H^*\left(H\left(Z_5\right)\right)+(-\rho)\left(X^{k+1}+\frac{\Lambda_5^k}{\rho}-Z_5\right)=0 \\ & Z_6^{k+1}=\min \left(\max \left(X^{k+1}+\frac{\Lambda_6^k}{\rho}, 0\right), 255\right) \\ & \beta D_1^*\left(X^{k+1}\right)-\rho\left(X^{k+1}+\frac{\Lambda_1^k}{\rho}-Z_1\right)=0 \end{aligned}$

$\beta D_2^*\left(X^{k+1}\right)-\rho\left(X^{k+1}+\frac{\Lambda_2^k}{\rho}-Z_2\right)=0$

$\beta D_3^*\left(X^{k+1}\right)-\rho\left(X^{k+1}+\frac{\Lambda_3^k}{\rho}-Z_3\right)=0$

$\beta D_4^*\left(X^{k+1}\right)-\rho\left(X^{k+1}+\frac{\Lambda_4^k}{\rho}-Z_4\right)=0$

In [ ]:
######3.5#####PF#####
using DataFrames, FFTW, Wavelets, Images, LinearAlgebra, Random, StatsBase,ImageView

Random.seed!(0)
img_path = "MRI_of_Human_Brain.jpg"
X = Float64.(load("MRI_of_Human_Brain.jpg"))
Omega = sample(collect(1:70144),21043,replace=false);
Omega = sort(Omega);

function PFOmega(X, Omega)
    F0 = fft(X)
    F0_vec = vec(F0);
    y = F0_vec[Omega];
    return y
end

function PFOmega_adjoint(y, Omega)
    Y = Complex.(zeros(70144,));
    m = 1
    for i in Omega
        Y[i] = y[m]
        m += 1
    end
    X = reshape(Y, (274, 256))
    y = real.(ifft(X))
    return y
end

In [ ]:
######3.5#####H#####
function WHaar(X)
    W = dwt(X, wavelet(WT.haar,WT.Filter));
    return W
end
   
function WHaar_adjoint(W)
    X = idwt(W, wavelet(WT.haar, WT.Filter))
    return X
end

In [28]:
######3.5#####D1D2D3D4#####
function D1(X)
    M, N = size(X)
    res = zeros((M-1)*N)
    m = 1
    for i in 2:M
        for j in 1:N
            res[m] = X[i, j] - X[i - 1, j]
            m += 1
        end
    end
    return res
end
   
function D1_adjoint(res,M,N)
    X = zeros(M,N)
    m = 1
    for i in 2:M
        for j in 1:N
            X[i, j] = X[i - 1, j] + res[m]
            m += 1
        end
    end
    return X
end
   
function D2(X)
    M, N = size(X)
    res = zeros(M*(N-1))
    m = 1
    for i in 1:M
        for j in 2:N
            res[m] = X[i, j] - X[i, j - 1]
            m += 1
        end
    end
    return res
end
   
function D2_adjoint(res,M,N)
    X = zeros(M,N)
    m = 1
    for i in 2:M
        for j in 1:N
            X[i, j] = X[i, j - 1] + res[m]
            m += 1
        end
    end
    return X
end
   
function D3(X)
    M, N = size(X)
    res = zeros((M-1) * (N-1))
    m = 1
    for i in 2:M
        for j in 2:N
            res[m] = X[i, j] - X[i - 1, j - 1]
            m += 1
        end
    end
    return res
end
   
function D3_adjoint(res,M,N)
    X = zeros(M,N)
    m = 1
    for i in 2:M
        for j in 1:N
            X[i, j] = X[i - 1, j - 1] + res[m]
            m += 1
        end
    end
    return X
end
   
function D4(X)
    M, N = size(X)
    res = zeros((M-1) * (N-1))
    m = 1
    for i in 2:M
        for j in 1:N
            res[m] = X[i, j] - X[i - 1, j + 1]
            m += 1
        end
    end
    return res
end
   
function D4_adjoint(res,M,N)
    X = zeros(M,N)
    m = 1
    for i in 2:M
        for j in 1:N
            X[i, j] = X[i - 1, j + 1] + res[m]
            m += 1
        end
    end
    return X
end

D4_adjoint (generic function with 1 method)